# <center>Lab Two Classification</center>
<center><font size = "4"> 2017-2018 California Department of Education Mathmematics Achievement</font></center>

##### <center>Create by An Nguyen, Andy Ho, Jodi Pafford</center>
<center> March 8, 2019</center>

# Total Points

# Data Preparation 1
Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis

In [18]:
import pandas as pd
import numpy as np

##Load original dataset.  No longer needed.
##The cleaning process is very computationally expensive therefore a the 'ranifall.csv' file was created for later use. 
#rainfall_original = pd.read_csv('weatherAus.csv') 

#Functions to find the average value using the bracketing values around the NaN's.  
    #For instance if a city's 'MinTemp' has 34, 32, NaN, NaN, 55 recorded 
    #the function will average 32 and 55 for the first NaN: (32+55)/2 = 43.5 
    #and average the above value and 55 for the second NaN: (43.5+55)/2 = 49.25
#Will only use values if they are from the same city.
#If NaN is the earliest timepoint for a given city the next timepoint with no NaN will be given instead of the mean.
#If NaN is the latest timepoint for a given city the previous timepoint with no NaN will be given instead of the mean.

def impute_by_city(cities,variables):
    for c in cities:
        #aPrse out observations from a single city.
        temp = rainfall[rainfall.Location == c]
        
        #Interate through all observations of the temp data file.
        i = min(temp.index)
        while i <= max(temp.index):
            for v in variables:
                #Check to see if there are values recorded for the variable, will pass over if all are NaN.
                if pd.isna(temp[v]).all():
                    pass
                
                #Check to see if a single value is NaN.
                elif pd.isna(temp[v][i]):
                    #Find the mean of bracketing values and impute into main dataframe.
                    temp[v][i] = find_mean(temp[v], i)
                    rainfall[v][i] = temp[v][i]
            i = i + 1       

#Find mean of bracketing values.
def find_mean(templist, index):
    #If NaN is earliest timepoint for the city take the next value that is not NaN.
    if index == min(templist.index): 
        return find_top(templist, index)
    
    #If latest timepoint for the city take the previous value that is not NaN.
    elif index == max(templist.index): 
        return find_bottom(templist, index)
    
    else:
        #Find previous non-NaN value.
        bottom = find_bottom(templist, index) 
        #Find next non-NaN value.
        top = find_top(templist, index) 
        
    #If current value is not from the latest timepoint for the city but there are no more non-NaN value recorded
    #after this value then the previous non-NaN value will be taken.
    if pd.isna(top): 
        return bottom
    

    else:
        mean = (top + bottom)/2
        return mean

#Find previous non-NaN value.
def find_bottom(templist, index):
    while pd.isna(templist[index-1]):
        index = index-1
    bottom = templist[index-1]
    return bottom

#Find next non-NaN value.
#If there are no more non-NaN values return the previous non-NaN value.
def find_top(templist, index):
    while pd.isna(templist[index+1]):
        index = index+1
        if index == max(templist.index):
            top = np.nan
            return top
    top = templist[index+1]
    return top   

In [19]:
##Code for first run data cleaning, no longer needed after 'rainfall.csv' was created at the end of cleaning process.

#rainfall = rainfall_original.copy()

##'RISK_MM' was used by creator of dataset to extrapolate response variable, 'RainTomorrow.'  Needs to be dropped to not 
##influence prediction.
#rainfall.drop(["RISK_MM"], axis=1, inplace=True)

##Drop any observation with no record of rainfall for the day.  Cannot be imputed.
#rainfall.dropna(subset=["RainToday"], inplace=True)

#Reset the Index of each observation to match it's iloc, get rid of gaps between Index integers.
#rainfall = rainfall.reset_index(drop=True)
#rainfall.info()

In [20]:
##can be skipped if rainfall.csv already generated!

##set the cardinal directions to degrees.
#directions = {'N':0, 'NNE':22.5, 'NE':45, 'NE':45, 'ENE':67.5, 'E':90, 'ESE':112.5, 'SE':135, 'SSE':157.5, 'S':180,\
#              'SSW':202.5, 'SW':225, 'WSW':247.5, 'W':270, 'WNW':292.5, 'NW':315, 'NNW':337.5}

##Replace cardianl direction to their corresponding degrees.
#rainfall = rainfall.replace(directions) 

#Get name of all cities in the data frame.
#cities = rainfall.Location.unique() 

#c_variables = []
#d_variables = []

##change 'Yes' and 'No' to 1 and 0 respectively.
#rainfall.RainToday = rainfall.RainToday=='Yes'
#rainfall.RainToday = rainfall.RainToday.astype(np.int)
#rainfall.RainTomorrow = rainfall.RainTomorrow=='Yes'
#rainfall.RainTomorrow = rainfall.RainTomorrow.astype(np.int)

##Find all variables with continous data type.
#for l in list(rainfall):
#    if (rainfall[l].dtypes == 'float64'):
#        c_variables.append(l)
#    else:
#        d_variables.append(l)

In [21]:
##can be skipped if rainfall.csv already generated! Very expensive, 'rainfall.csv' can be uploaded from working directory

##Impute values to NaN's and save to csv file for later use.
#impute_by_city(cities, c_variables)
#rainfall.to_csv("rainfall.csv", sep=',', index=True)

In [22]:
##load pre-generated rainfall.csv file.
rainfall = pd.read_csv('rainfall.csv', index_col=0) 

#Variables 'Evaporation' and 'Sunshine' contained many missing values, too many to be imputed.
rainfall = rainfall.drop(['Evaporation', 'Sunshine'], axis = 1)

#Get name of all cities in the data frame.
l = list(rainfall.Location.unique())

#Drop all observations with NaN's.  These are values that could not be imputed using the above code.
rainfall.dropna(subset = list(rainfall), inplace = True)

#List all cities that were dropped
for i in l:
    if i not in rainfall.Location.unique():
        print(i)
        
#'Date' and 'Location' variables not needed for prediction. 
rainfall = rainfall.drop(['Date', 'Location'], axis = 1) 

BadgerysCreek
Newcastle
NorahHead
Penrith
Tuggeranong
MountGinini
Nhil
Dartmoor
GoldCoast
Adelaide
Albany
Witchcliffe
SalmonGums
Walpole


# Data Preparation 2
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

# Modeling and Evaluation 1
Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

# Modeling and Evaluation 2
Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time. 

In [23]:
from sklearn.model_selection import ShuffleSplit as ss
   
#Split our data into training and testing sets, 80% of data will be in the training set and 20% the testing set.
#Data will be process this way 5 times, value can be change per user's judgement.  It is recommended that number
#of iterations be at least 2 so that standard deviations can be computed.
num_cv_iterations = 1
num_instances = len(y)
cv_object = ss(n_splits=num_cv_iterations, test_size  = 0.2)

# Modeling and Evaluation 3
Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

* Task 1 Classification: Predict if it will rain the next day or not.
* Task 2 Regression: Compute how much it rained that day.

## Task 1 Classification:  Predit if it will rain the next day or not.

In [35]:
from sklearn import metrics as mt
from sklearn.preprocessing import StandardScaler as sts
from sklearn.linear_model import LogisticRegression as lr
from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.naive_bayes import GaussianNB as gnb
import time

#Assign values to response variable, y, and explanatory variables, x.
temp_rainfall = rainfall.copy()
if 'RainTomorrow' in temp_rainfall:
    #Response variable is 'RainTomorrow'
    y = temp_rainfall['RainTomorrow'].values
    
    #Remove response variable from dataframe
    del temp_rainfall['RainTomorrow']
    
    #Everything else is the explanatory variables used in prediction.
    x = temp_rainfall.values
    
scl_obj = sts()
accuracy_dict = {}

#Split the data into training and testing set 5 different ways, iterate through each way.
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(x,y)):
    
    #Standardize the explanatory variables of the training and testing sets' means to be around 0 with a standard deviation of 1.  
    #Each value is subtracted from the mean and divided by the standard deviation of the whole dataset.
    scl_obj.fit(x[train_indices])
    X_train_scaled = scl_obj.transform(x[train_indices])
    X_test_scaled = scl_obj.transform(x[test_indices])

    #K Nearest Neighbors, all neighbors are treated the same.  Number of neighbors range from 1 to 10.
    for K in range(1, 11):
        t0=time.time()
        knn_clf = knn(n_neighbors=K, weights='uniform')
        knn_clf.fit(X_train_scaled,y[train_indices])
        y_hat = knn_clf.predict(X_test_scaled)
        acc = mt.accuracy_score(y[test_indices],y_hat)
        accuracy_dict.update({'KNN with %d neighbors with \'uniform\' weights' %(K): acc})
        print('Accuracy of KNN with %d neighbors with \'uniform\' weights is: %.5f'%(K,acc))
        print ("Time to Run:", time.time()-t0)
        
    #K Nearest Neighbors, closer neighbors are given more weight.  Number of neighbors range from 1 to 10.
    for K in range(1, 11):
        t0=time.time()
        knn_clf = knnc(n_neighbors=K, weights='distance')
        knn_clf.fit(X_train_scaled,y[train_indices])
        y_hat = knn_clf.predict(X_test_scaled)
        acc = mt.accuracy_score(y[test_indices],y_hat)
        accuracy_dict.update({'KNN with %d neighbors with \'distance\' weights' %(K): acc})
        print('Accuracy of KNN with %d neighbors with \'distance\' weights is: %.5f'%(K,acc))
        print ("Time to Run:", time.time()-t0)
           
    #Gaussian Naive Bayes
    t0=time.time()
    gnb_clf = gnb()
    gnb_clf.fit(X_train_scaled,y[train_indices])
    y_hat = gnb_clf.predict(X_test_scaled)
    acc = mt.accuracy_score(y[test_indices],y_hat)
    accuracy_dict.update({'Gausian Naive Bayes': acc})
    print('Accuracy of Gausian Naive Bayes is: %.5f' %(acc))
    print ("Time to Run:", time.time()-t0)
    
    
    #Perform Logistic Regression, cost (inverse of regulation strength) range from 1 to 10 and l1 penalty.
    for cost in range(1,11):
        t0=time.time()
        lr_clf = lr(penalty='l1', C=cost)
        lr_clf.fit(X_train_scaled,y[train_indices])
        y_hat = lr_clf.predict(X_test_scaled)
        acc = mt.accuracy_score(y[test_indices],y_hat)
        accuracy_dict.update({'Logistic Regression \'l1\' penalty with cost %d' %(cost): acc})
        print('Accuracy of Logistic Regression \'l1\' penalty with cost %d is: %.5f' %(cost, acc))
        print ("Time to Run:", time.time()-t0)
    
    #Perform Logistic Regression, cost (inverse of regulation strength) range from 1 to 10 and l2 penalty.
    for cost in range(1,11):
        t0=time.time()
        lr_clf = lr(penalty='l2', C=cost)
        lr_clf.fit(X_train_scaled,y[train_indices])
        y_hat = lr_clf.predict(X_test_scaled)
        acc = mt.accuracy_score(y[test_indices],y_hat)
        accuracy_dict.update({'Logistic Regression \'l2\' penalty with cost %d' %(cost): acc})
        print('Accuracy of Logistic Regression \'l2\' penalty with cost %d is: %.5f' %(cost, acc))
        print ("Time to Run:", time.time()-t0)

print()
print('The maximum accuracy of %.5f is achieved with %s' %(max(accuracy_dict.items())[1], max(accuracy_dict.items())[0]))

Accuracy of KNN with 1 neighbors with 'uniform' weights is: 0.80731
Time to Run: 18.088132619857788
Accuracy of KNN with 2 neighbors with 'uniform' weights is: 0.82991
Time to Run: 21.955844402313232
Accuracy of KNN with 3 neighbors with 'uniform' weights is: 0.83302
Time to Run: 24.111968755722046
Accuracy of KNN with 4 neighbors with 'uniform' weights is: 0.83973
Time to Run: 27.3185977935791
Accuracy of KNN with 5 neighbors with 'uniform' weights is: 0.84284
Time to Run: 27.238258838653564
Accuracy of KNN with 6 neighbors with 'uniform' weights is: 0.84531
Time to Run: 28.14395046234131
Accuracy of KNN with 7 neighbors with 'uniform' weights is: 0.84792
Time to Run: 28.96383023262024
Accuracy of KNN with 8 neighbors with 'uniform' weights is: 0.84916
Time to Run: 29.89581799507141
Accuracy of KNN with 9 neighbors with 'uniform' weights is: 0.85049
Time to Run: 31.055959701538086
Accuracy of KNN with 10 neighbors with 'uniform' weights is: 0.85093
Time to Run: 32.8038604259491
Accura

## Task 2 Regression: Compute how much it rained that day.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor as knnr
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.svm import SVR

if 'Rainfall' in temp_rainfall:
    #Response variable is 'RainTomorrow'
    y = temp_rainfall['Rainfall'].values
    
    #Remove response variable from dataframe
    del temp_rainfall['Rainfall']
    
    #Everything else is the explanatory variables used in prediction.
    x = temp_rainfall.values
    
scl_obj = sts()

#Split the data into training and testing set 5 different ways, iterate through each way.
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(x,y)):
    
    #Standardize the explanatory variables of the training and testing sets' means to be around 0 with a standard deviation of 1.  
    #Each value is subtracted from the mean and divided by the standard deviation of the whole dataset.
    scl_obj.fit(x[train_indices])
    X_train_scaled = scl_obj.transform(x[train_indices])
    X_test_scaled = scl_obj.transform(x[test_indices])
    
    rmse_val = []
    for K in range(1,31):
        t0=time.time()
        knnr_clf = knnr(n_neighbors=K, weights='uniform')
        knnr_clf.fit(X_train_scaled,y[train_indices])
        y_hat = knnr_clf.predict(X_test_scaled)
        error = np.sqrt(mse(y[test_indices],y_hat))
        rmse_val.append(error)
        print('RMSE of KNN with %d neighbors with \'uniform\' weights is: %.5f'%(K,error))
        print ("Time to Run:", time.time()-t0)
    #plotting the rmse values against k values
    curve = pd.DataFrame(rmse_val) #elbow curve 
    curve.plot()
    print('The number of neighbors with the lowest RMSE is %d, RMSE of %.5f.' %(rmse_val.index(min(rmse_val))+1,min(rmse_val)))
    
    rmse_val = []
    for K in range(1,31):
        t0=time.time()
        knnr_clf = knnr(n_neighbors=K, weights='distance')
        knnr_clf.fit(X_train_scaled,y[train_indices])
        y_hat = knnr_clf.predict(X_test_scaled)
        error = np.sqrt(mse(y[test_indices],y_hat))
        rmse_val.append(error)
        print('RMSE of KNN with %d neighbors with \'distance\' weights is: %.5f'%(K,error))
        print ("Time to Run:", time.time()-t0)
    #plotting the rmse values against k values
    curve = pd.DataFrame(rmse_val) #elbow curve 
    curve.plot()
    print('The number of neighbors with the lowest RMSE is %d, RMSE of %.5f.' %(rmse_val.index(min(rmse_val))+1,min(rmse_val)))
    
    rmse_val = []
    for K in range(2,10):
        t0=time.time()
        dtr_clf = dtr(max_depth = K)
        dtr_clf.fit(X_train_scaled,y[train_indices])
        y_hat = knnr_clf.predict(X_test_scaled)        
        error = np.sqrt(mse(y[test_indices],y_hat))
        rmse_val.append(error)
        print('RMSE of Decision Tree with %d depth is: %.5f'%(K,error))
        print ("Time to Run:", time.time()-t0)
    #plotting the rmse values against k values
    curve = pd.DataFrame(rmse_val) #elbow curve 
    curve.plot()
    print('The depth with the lowest RMSE is %d, RMSE of %.5f.' %(rmse_val.index(min(rmse_val))+1,min(rmse_val)))

    rmse_val = []
    for e in np.arange(.1,1,.1):
        t0=time.time()
        svr_clf = SVR(epsilon=e)
        svr_clf.fit(X_train_scaled,y[train_indices])
        y_hat = svr_clf.predict(X_test_scaled)
        error = np.sqrt(mse(y[test_indices],y_hat))
        rmse_val.append(error)
        print('RMSE of Support Vector Regression with epsilon = %.1f is: %.5f'%(e,error))
        print ("Time to Run:", time.time()-t0)
    #plotting the rmse values against k values
    curve = pd.DataFrame(rmse_val) #elbow curve 
    curve.plot()
    print('The depth with the lowest RMSE is %d, RMSE of %.5f.' %(rmse_val.index(min(rmse_val))+1,min(rmse_val)))

# Modeling and Evaluation 4
Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model. 

# Modeling and Evaluation 5
Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

# Modeling and Evaluation 6
Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task. 

# Deployment
How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

# Exceptional Work
You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm